# Spectrogram

In [2]:
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Spectrogram
import numpy as np

print('Keras version: {}'.format(keras.__version__))
print('Keras backend: {}'.format(keras.backend._backend))
print('Keras image dim ordering: {}'.format(keras.backend.image_dim_ordering()))
print('Kapre version: {}'.format(kapre.__version__))

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Spectrogram(n_dft=512, n_hop=256, input_shape=src.shape, 
          return_decibel_spectrogram=True, power_spectrogram=2.0, 
          trainable_kernel=False, name='static_stft'))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])


Keras version: 1.2.1
Keras backend: theano
Keras image dim ordering: th
Kapre version: 0.0.3
________________________________________________________________________________
Layer (type)              Output Shape              Param #     Connected to    
static_stft (Spectrogram) (None, 2, 257, 173)       263168      spectrogram_inpu
Total params: 263,168
Trainable params: 0
Non-trainable params: 263,168
________________________________________________________________________________


# Spectrogram (trainable)

In [6]:
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Spectrogram
import numpy as np

print('Keras version: {}'.format(keras.__version__))
print('Keras backend: {}'.format(keras.backend._backend))
print('Keras image dim ordering: {}'.format(keras.backend.image_dim_ordering()))
print('Kapre version: {}'.format(kapre.__version__))

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Spectrogram(n_dft=2048, n_hop=1024, input_shape=src.shape, 
          return_decibel_spectrogram=True, power_spectrogram=2.0, 
          trainable_kernel=True, name='trainable_stft'))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])

Keras version: 1.2.1
Keras backend: theano
Keras image dim ordering: th
Kapre version: 0.0.3
________________________________________________________________________________
Layer (type)              Output Shape              Param #     Connected to    
trainable_stft (Spectrogr (None, 2, 1025, 44)       4198400     spectrogram_inpu
Total params: 4,198,400
Trainable params: 4,198,400
Non-trainable params: 0
________________________________________________________________________________


# Melspectrogram

In [7]:
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Melspectrogram
import numpy as np

print('Keras version: {}'.format(keras.__version__))
print('Keras backend: {}'.format(keras.backend._backend))
print('Keras image dim ordering: {}'.format(keras.backend.image_dim_ordering()))
print('Kapre version: {}'.format(kapre.__version__))

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Melspectrogram(sr=16000, n_mels=128, 
          n_dft=512, n_hop=256, input_shape=src.shape, 
          return_decibel_spectrogram=True,
          trainable_kernel=True, name='melgram'))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])

Keras version: 1.2.1
Keras backend: theano
Keras image dim ordering: th
Kapre version: 0.0.3
________________________________________________________________________________
Layer (type)              Output Shape              Param #     Connected to    
melgram (Melspectrogram)  (None, 2, 128, 173)       296064      melspectrogram_i
Total params: 296,064
Trainable params: 263,168
Non-trainable params: 32,896
________________________________________________________________________________


# AdditiveNoise

In [8]:
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Melspectrogram
from kapre.augmentation import AdditiveNoise
import numpy as np

print('Keras version: {}'.format(keras.__version__))
print('Keras backend: {}'.format(keras.backend._backend))
print('Keras image dim ordering: {}'.format(keras.backend.image_dim_ordering()))
print('Kapre version: {}'.format(kapre.__version__))

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Melspectrogram(sr=16000, n_mels=128, 
          n_dft=512, n_hop=256, input_shape=src.shape, 
          return_decibel_spectrogram=True,
          trainable_kernel=False, name='melgram'))
model.add(AdditiveNoise(power=0.2))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])

Keras version: 1.2.1
Keras backend: theano
Keras image dim ordering: th
Kapre version: 0.0.3
________________________________________________________________________________
Layer (type)              Output Shape              Param #     Connected to    
melgram (Melspectrogram)  (None, 2, 128, 173)       296064      melspectrogram_i
________________________________________________________________________________
additivenoise_1 (Additive (None, 2, 128, 173)       0           melgram[0][0]   
Total params: 296,064
Trainable params: 0
Non-trainable params: 296,064
________________________________________________________________________________


# Stft

In [9]:
import keras
import kapre
from keras.models import Sequential
from kapre.stft import Stft
import numpy as np

print('Keras version: {}'.format(keras.__version__))
print('Keras backend: {}'.format(keras.backend._backend))
print('Keras image dim ordering: {}'.format(keras.backend.image_dim_ordering()))
print('Kapre version: {}'.format(kapre.__version__))

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Stft(n_fft=256, n_hop=64, return_decibel_stft=True,
              input_shape=src.shape))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])

Keras version: 1.2.1
Keras backend: theano
Keras image dim ordering: th
Kapre version: 0.0.3
________________________________________________________________________________
Layer (type)              Output Shape              Param #     Connected to    
stft_4 (Stft)             (None, 129, 686, 2)       0           stft_input_4[0][
Total params: 0
Trainable params: 0
Non-trainable params: 0
________________________________________________________________________________
